# Treinamento em Lote de Modelos GRU para Capitais Estaduais

Este notebook estende a arquitetura do modelo GRU simples para treinar um modelo de previsão de morbidade circulatória para cada capital de estado no Brasil.

O fluxo de trabalho é projetado para ser totalmente automatizado e modular:

1.  **Configuração Centralizada:** Uma lista de códigos de município (CD_MUN) para as capitais é definida, juntamente com todos os outros parâmetros do modelo e do treinamento.
2.  **Loop de Treinamento em Lote:** O notebook itera sobre cada capital. Em cada iteração, ele executa o pipeline completo:
    *   **Carregamento e Filtragem:** Carrega o conjunto de dados completo e filtra os dados para a capital atual.
    *   **Pré-processamento:** Converte a coluna de data, define o índice e limpa a série temporal.
    *   **Preparação dos Dados:** Utiliza `prepare_data_for_model` para criar os conjuntos de treinamento, validação e teste, juntamente com o scaler.
    *   **Construção e Treinamento:** Constrói e treina um novo modelo GRU do zero para a cidade.
    *   **Avaliação:** Avalia o modelo treinado nos dados de teste.
    *   **Visualização e Salvamento:** Gera as mesmas visualizações detalhadas (previsão combinada de validação/teste e gráfico de dispersão) que no notebook anterior e salva todos os resultados (métricas, previsões e gráficos) em um subdiretório específico para a execução.

O objetivo é criar um sistema robusto que possa ser executado para gerar e salvar sistematicamente os resultados de previsão para várias cidades, mantendo a organização e a reprodutibilidade.

In [1]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import logging

# --- Configuração do Logging ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --- Adicionar o Diretório Raiz ao sys.path ---
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    logging.info(f"Adicionado {project_root} ao sys.path")

# --- Importar Módulos Customizados ---
from src.preprocessing import load_city_data, prepare_data_for_model, filter_city, clean_timeseries
from src.models import build_gru
from src.train import train_model, evaluate_model, save_metrics, save_predictions
from src.utils import plot_training_history, plot_actual_vs_predicted_scatter

# --- Sementes para Reprodutibilidade ---
np.random.seed(42)
tf.random.set_seed(42)

logging.info("Módulos importados e configuração inicial completa.")

2025-06-24 21:54:29,796 - INFO - Adicionado c:\Users\pedro\OneDrive - Unesp\Documentos\GitHub\cities-models\cities-models ao sys.path
2025-06-24 21:54:30,035 - INFO - Módulos importados e configuração inicial completa.
2025-06-24 21:54:30,035 - INFO - Módulos importados e configuração inicial completa.


In [2]:
# =============================================================================
# Célula de Configuração do Lote
# =============================================================================

# --- Lista de Capitais Estaduais (CD_MUN e Nomes) ---
STATE_CAPITALS = {
    1100205: "Porto_Velho", 1200401: "Rio_Branco", 1302603: "Manaus", 1400100: "Boa_Vista",
    1501402: "Belem", 1600303: "Macapa", 1721000: "Palmas", 2111300: "Sao_Luis",
    2211001: "Teresina", 2304400: "Fortaleza", 2408102: "Natal", 2507507: "Joao_Pessoa",
    2611606: "Recife", 2704302: "Maceio", 2800308: "Aracaju", 2927408: "Salvador",
    3106200: "Belo_Horizonte", 3205309: "Vitoria", 3304557: "Rio_de_Janeiro",
    3550308: "Sao_Paulo", 4106902: "Curitiba", 4205407: "Florianopolis",
    4314902: "Porto_Alegre", 5002704: "Campo_Grande", 5103403: "Cuiaba",
    5208707: "Goiania", 5300108: "Brasilia"
}

# --- Seleção dos Dados ---
TARGET_COLUMN = 'target'
FEATURE_COLUMNS = ['target']

# --- Divisão dos Dados ---
TEST_SIZE = 52
VAL_SIZE = 52

# --- Parâmetros do Modelo ---
SEQUENCE_LENGTH = 12
FORECAST_HORIZON = 1

# --- Parâmetros de Treinamento ---
EPOCHS = 200
BATCH_SIZE = 32
PATIENCE = 15

# --- Arquitetura do Modelo ---
GRU_UNITS = 32
LOSS_FUNCTION = 'mae'

# --- Caminhos ---
DATA_PATH = '../data/df_base_morb_circ.csv'
RUN_NAME = "gru_batch_state_capitals_morb_circ"
RESULTS_DIR = os.path.join('results', RUN_NAME)
os.makedirs(RESULTS_DIR, exist_ok=True)

# --- Configuração do Log de Execução ---
log_file_path = os.path.join(RESULTS_DIR, 'execution_log.txt')
file_handler = logging.FileHandler(log_file_path, mode='w')
file_handler.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))
logging.getLogger().addHandler(file_handler)

logging.info("--- Configuração do Experimento em Lote ---")
logging.info(f"Execução: {RUN_NAME}")
logging.info(f"Total de cidades a serem processadas: {len(STATE_CAPITALS)}")
logging.info(f"Resultados serão salvos em: {RESULTS_DIR}")
logging.info("-" * 40)

2025-06-24 21:54:30,047 - INFO - --- Configuração do Experimento em Lote ---
2025-06-24 21:54:30,048 - INFO - Execução: gru_batch_state_capitals_morb_circ
2025-06-24 21:54:30,049 - INFO - Total de cidades a serem processadas: 27
2025-06-24 21:54:30,050 - INFO - Resultados serão salvos em: results\gru_batch_state_capitals_morb_circ
2025-06-24 21:54:30,051 - INFO - ----------------------------------------
2025-06-24 21:54:30,048 - INFO - Execução: gru_batch_state_capitals_morb_circ
2025-06-24 21:54:30,049 - INFO - Total de cidades a serem processadas: 27
2025-06-24 21:54:30,050 - INFO - Resultados serão salvos em: results\gru_batch_state_capitals_morb_circ
2025-06-24 21:54:30,051 - INFO - ----------------------------------------


In [3]:
# =============================================================================
# Loop de Treinamento em Lote
# =============================================================================

# Carregar o dataset completo uma vez
try:
    df_full = load_city_data(DATA_PATH)
    logging.info(f"Dataset completo carregado com sucesso: {df_full.shape}")
except Exception as e:
    logging.error(f"Falha ao carregar o dataset principal. Encerrando a execução. Erro: {e}")
    df_full = None

if df_full is not None:
    for cd_mun, city_name in STATE_CAPITALS.items():
        logging.info(f"===== Iniciando processamento para {city_name} ({cd_mun}) =====")

        try:
            # --- 1. Filtrar e Preparar Dados ---
            df_city = filter_city(df_full, cd_mun=cd_mun)
            if df_city.empty:
                logging.warning(f"Nenhum dado encontrado para {city_name} ({cd_mun}). Pulando.")
                continue

            df_city['week'] = pd.to_datetime(df_city['week'])
            df_city = df_city.set_index('week').sort_index()
            df_city = clean_timeseries(df_city, target_column=TARGET_COLUMN)
            logging.info(f"Dados para {city_name} filtrados e limpos: {df_city.shape}")

            # --- 2. Preparar Dados para o Modelo ---
            data_dict = prepare_data_for_model(
                df=df_city,
                target_column=TARGET_COLUMN,
                feature_columns=FEATURE_COLUMNS.copy(),
                sequence_length=SEQUENCE_LENGTH,
                forecast_horizon=FORECAST_HORIZON,
                test_size=TEST_SIZE,
                val_size=VAL_SIZE,
            )
            X_train, y_train = data_dict['X_train'], data_dict['y_train']
            X_val, y_val = data_dict['X_val'], data_dict['y_val']
            X_test, y_test = data_dict['X_test'], data_dict['y_test']
            scaler = data_dict['scaler']
            original_val_df = data_dict['original_val_df']
            original_test_df = data_dict['original_test_df']
            logging.info(f"Dados preparados para o modelo. Treino: {X_train.shape}, Validação: {X_val.shape}, Teste: {X_test.shape}")

            # --- 3. Construir e Treinar o Modelo ---
            tf.keras.backend.clear_session()
            np.random.seed(42)
            tf.random.set_seed(42)
            
            input_shape = X_train.shape[1:]
            model = build_gru(input_shape=input_shape, units=GRU_UNITS, loss=LOSS_FUNCTION)
            
            history = train_model(
                model=model, X_train=X_train, y_train=y_train, X_val=X_val, y_val=y_val,
                epochs=EPOCHS, batch_size=BATCH_SIZE, patience=PATIENCE, verbose=0
            )
            logging.info(f"Modelo treinado para {city_name}. Perda final de validação: {history.history['val_loss'][-1]:.4f}")

            # --- 4. Avaliar o Modelo ---
            y_pred_normalized = model.predict(X_test)
            metrics = evaluate_model(
                y_true=y_test, y_pred=y_pred_normalized, scaler=scaler,
                target_idx=FEATURE_COLUMNS.index(TARGET_COLUMN)
            )
            y_true_denorm = metrics['y_true_denorm']
            y_pred_denorm = metrics['y_pred_denorm']
            logging.info(f"Métricas de teste para {city_name}: MAE={metrics['mae']:.4f}, RMSE={metrics['rmse']:.4f}, R²={metrics['r2']:.4f}")

            # --- 5. Visualizar e Salvar Resultados ---
            # Gerar previsões de validação para o plot
            y_val_pred_normalized = model.predict(X_val)
            val_metrics = evaluate_model(
                y_true=y_val, y_pred=y_val_pred_normalized, scaler=scaler,
                target_idx=FEATURE_COLUMNS.index(TARGET_COLUMN)
            )
            y_val_true_denorm = val_metrics['y_true_denorm']
            y_val_pred_denorm = val_metrics['y_pred_denorm']

            # Combinar dados para o plot
            combined_true = np.concatenate([y_val_true_denorm, y_true_denorm])
            combined_pred = np.concatenate([y_val_pred_denorm, y_pred_denorm])
            val_dates = original_val_df.index[-len(y_val_true_denorm):]
            test_dates = original_test_df.index[-len(y_true_denorm):]
            combined_dates = val_dates.union(test_dates)

            # Plot combinado
            fig_combined, ax = plt.subplots(figsize=(18, 6))
            ax.plot(combined_dates, combined_true, 'o-', color='royalblue', label='Real')
            ax.plot(combined_dates, combined_pred, '.-', color='red', label='Previsão')
            test_start_date = test_dates[0]
            ax.axvline(x=test_start_date, color='green', linestyle='--', label=f'Início do Teste')
            ax.set_title(f'{city_name} - Previsão (Validação + Teste)', fontsize=16)
            ax.legend()
            ax.grid(True)
            fig_combined.savefig(os.path.join(RESULTS_DIR, f'{city_name}_combined_forecast.png'), dpi=300)
            plt.close(fig_combined)

            # Plot de dispersão
            fig_scatter = plot_actual_vs_predicted_scatter(
                y_true=y_true_denorm, y_pred=y_pred_denorm,
                title=f'{city_name} - Teste: Reais vs. Previstos', metrics=metrics
            )
            fig_scatter.savefig(os.path.join(RESULTS_DIR, f'{city_name}_scatter_plot.png'), dpi=300)
            plt.close(fig_scatter)
            logging.info(f"Gráficos salvos para {city_name}")

            # --- 6. Salvar Métricas e Predições ---
            save_predictions(
                y_true=y_true_denorm, y_pred=y_pred_denorm, dates=test_dates,
                city_name=city_name, model_name='gru_simple', output_dir=RESULTS_DIR
            )
            params_to_save = {'cd_mun': cd_mun, 'sequence_length': SEQUENCE_LENGTH, 'epochs': EPOCHS, 'gru_units': GRU_UNITS}
            metrics_to_save = {k: v for k, v in metrics.items() if k in ['mae', 'rmse', 'r2']}
            save_metrics(
                metrics=metrics_to_save, city_name=city_name, model_name='gru_simple',
                output_dir=RESULTS_DIR, params=params_to_save
            )
            logging.info(f"Métricas e previsões salvas para {city_name}")

        except ValueError as ve:
            logging.error(f"Erro de valor ao processar {city_name} ({cd_mun}): {ve}. Pulando.")
        except Exception as e:
            logging.error(f"Uma exceção inesperada ocorreu ao processar {city_name} ({cd_mun}): {e}", exc_info=True)
        
        logging.info(f"===== Processamento para {city_name} concluído =====\n")

    logging.info("--- Treinamento em lote concluído para todas as cidades ---")

2025-06-24 21:55:19,788 - INFO - Dataset completo carregado com sucesso: (6344064, 11)
2025-06-24 21:55:19,790 - INFO - ===== Iniciando processamento para Porto_Velho (1100205) =====
2025-06-24 21:55:19,790 - INFO - ===== Iniciando processamento para Porto_Velho (1100205) =====
2025-06-24 21:55:19,815 - INFO - Dados para Porto_Velho filtrados e limpos: (1152, 10)
2025-06-24 21:55:19,827 - INFO - Dados preparados para o modelo. Treino: (1036, 12, 1), Validação: (40, 12, 1), Teste: (40, 12, 1)
2025-06-24 21:55:19,815 - INFO - Dados para Porto_Velho filtrados e limpos: (1152, 10)
2025-06-24 21:55:19,827 - INFO - Dados preparados para o modelo. Treino: (1036, 12, 1), Validação: (40, 12, 1), Teste: (40, 12, 1)


2025-06-24 21:56:59,521 - WARNING - From c:\Users\pedro\OneDrive - Unesp\Documentos\GitHub\treinamento_clusters_hpc\.conda\lib\site-packages\keras\src\backend\common\global_state.py:82: The name tf.reset_default_graph is deprecated. Please use tf.compat.v1.reset_default_graph instead.

2025-06-24 21:57:13,483 - INFO - Modelo treinado para Porto_Velho. Perda final de validação: 0.1035
2025-06-24 21:57:13,483 - INFO - Modelo treinado para Porto_Velho. Perda final de validação: 0.1035


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step


2025-06-24 21:57:13,724 - INFO - Métricas de teste para Porto_Velho: MAE=0.7232, RMSE=0.9870, R²=0.3304


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step


...

2025-06-24 22:04:17,820 - INFO - Gráficos salvos para Goiania
2025-06-24 22:04:17,824 - INFO - Métricas e previsões salvas para Goiania
2025-06-24 22:04:17,825 - INFO - ===== Processamento para Goiania concluído =====

2025-06-24 22:04:17,825 - INFO - ===== Iniciando processamento para Brasilia (5300108) =====
2025-06-24 22:04:17,834 - INFO - Dados para Brasilia filtrados e limpos: (1152, 10)
2025-06-24 22:04:17,824 - INFO - Métricas e previsões salvas para Goiania
2025-06-24 22:04:17,825 - INFO - ===== Processamento para Goiania concluído =====

2025-06-24 22:04:17,825 - INFO - ===== Iniciando processamento para Brasilia (5300108) =====
2025-06-24 22:04:17,834 - INFO - Dados para Brasilia filtrados e limpos: (1152, 10)
2025-06-24 22:04:17,842 - INFO - Dados preparados para o modelo. Treino: (1036, 12, 1), Validação: (40, 12, 1), Teste: (40, 12, 1)
2025-06-24 22:04:17,842 - INFO - Dados preparados para o modelo. Treino: (1036, 12, 1), Validação: (40, 12, 1), Teste: (40, 12, 1)
2025-06-

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step


2025-06-24 22:04:29,810 - INFO - Métricas de teste para Brasilia: MAE=1.4185, RMSE=1.9908, R²=0.6522


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step


2025-06-24 22:04:30,374 - INFO - Gráficos salvos para Brasilia
2025-06-24 22:04:30,378 - INFO - Métricas e previsões salvas para Brasilia
2025-06-24 22:04:30,379 - INFO - ===== Processamento para Brasilia concluído =====

2025-06-24 22:04:30,380 - INFO - --- Treinamento em lote concluído para todas as cidades ---
2025-06-24 22:04:30,378 - INFO - Métricas e previsões salvas para Brasilia
2025-06-24 22:04:30,379 - INFO - ===== Processamento para Brasilia concluído =====

2025-06-24 22:04:30,380 - INFO - --- Treinamento em lote concluído para todas as cidades ---


## Conclusão do Treinamento em Lote

Este notebook executou com sucesso o treinamento, avaliação e salvamento de resultados para um modelo GRU em múltiplas capitais estaduais.

- **Automação:** O processo foi totalmente automatizado, iterando sobre uma lista predefinida de cidades.
- **Modularidade:** O código aproveitou as funções do diretório `src/`, mantendo o notebook limpo e focado na orquestração do fluxo de trabalho em lote.
- **Organização dos Resultados:** Todos os artefatos gerados (gráficos, métricas e previsões) para cada cidade foram salvos em um diretório de execução claramente nomeado, garantindo que os resultados sejam fáceis de encontrar e analisar.
- **Tratamento de Erros:** O loop inclui tratamento de exceções para garantir que a falha no processamento de uma cidade não interrompa todo o processo.

Os resultados agora estão disponíveis no diretório `results/gru_batch_state_capitals_morb_circ/` para análise detalhada.